In [ ]:
import pandas as pd
from minio import Minio
from minio.error import S3Error

# Configurações do Minio
minio_host = 'minio.ifpbapps.online'
minio_access_key = "VQdTNNVXZZLXNMRnV4RDFoQ2MtZGFSRjN3Q0Z2Z"
minio_secret_key = "pMnQ4ZkktaU9UdlRYX1c!3S1lHMkVpajU5LTRLTHl1NwbmRDVmdiWW9RaGJvZmFWSkF5YmZ"
minio_bucket = 'raios'

# Conectar ao Minio
minio_client = Minio(minio_host, access_key=minio_access_key, secret_key=minio_secret_key, secure=True)

# Função para baixar arquivo do Minio e carregar como DataFrame
def download_and_read_csv(minio_client, bucket, file_path, encoding='latin-1', delimiter=','):
    try:
        # Baixar o arquivo do Minio
        data = minio_client.get_object(bucket, file_path)
        
        # Ler o CSV como bytes
        data_bytes = data.read()

        # Decodificar os bytes para um DataFrame
        df = pd.read_csv(io.BytesIO(data_bytes), encoding=encoding, delimiter=delimiter)
        
        return df
    except S3Error as err:
        print(err)
        return None

# Lista de arquivos no Minio
all_files = minio_client.list_objects(minio_bucket, prefix='Consolidado Raios 2023_')
files = [file.object_name for file in all_files]

# Lista para armazenar DataFrames
dfs = []

# Iterar sobre os arquivos e baixar do Minio
for file in files:
    df = download_and_read_csv(minio_client, minio_bucket, file)
    if df is not None:
        dfs.append(df)

# Concatenar os DataFrames
Descargas_Energisa = pd.concat(dfs, ignore_index=True)

# Carregar os DataFrames Cod_IBGE e Ocorrencias diretamente do Minio
Cod_IBGE = download_and_read_csv(minio_client, minio_bucket, 'cod_ibge.xls', encoding='latin-1')
Ocorrencias = download_and_read_csv(minio_client, minio_bucket, 'ocorrencias-emergenciais-rede-distribuicao-2023.csv', encoding='latin-1')